# Simple PINN for an elastic plate with an elliptical hole 

## Geometry

We want to model a quarter of a plate with an elliptical hole. The domain itself is represented by collocation points, the boundaries are represented by uniformly sampled points along the perimeter.

In [ ]:
import torch
from torch.optim.lr_scheduler import StepLR
import numpy as np
from scipy.stats import qmc
from plotly.express.colors import sequential
import plotly.graph_objects as go
import plotly.figure_factory as ff
from tqdm import tqdm
import wandb
import matplotlib.pyplot as plt

torch.set_default_dtype(torch.float64)
from global_constants import L, R, B0, EPS0, MU, LBD, RATIO

torch.set_default_dtype(torch.float64)

# Elliptical axis in x direction
Rx = 0.3
# Number of collocation points
N = 25

# Epochs
epochs = 8000
# second epochs for second optimizer
epochs2 = 4000
# Batch size
batch_size = 64
# Leanring rate
lr = 0.001
# Scheduler step width
scheduler_step = 2000 
# Gamma factor of scheduler
scheduler_gamma = 0.5

# Number of hidden neurons
hn = 50

# Weight of PDE loss
W_PDE = 0.00
# Weight of Neumann loss
W_NEU = 0.0
# Weight of data losses
W_STRESS = 0.0
W_DISP = 0.0

# Weights and Biases
#wandb.login( key = "9351c110385f8824931c4f4479b39f325a89051e" )


wandb.init(
    project="pinn_shape_optimization",
    entity="ddped",
    config={
        "Rx": Rx,
        "N_collo": N,
        "epochs": epochs,
        "batch_size": batch_size,
        "learning_rate": lr,
        "scheduler_step": scheduler_step,
        "scheduler_gamma": scheduler_gamma,
        "hidden_neurons": hn,
        "n_layers": 5,
        "data_type": "Stress",
        "W_DATA_STRESS": W_STRESS,
        "W_DATA_DISP": W_DISP,
        "W_NEUMANN": W_NEU,
        "W_PDE": W_PDE,
    },
)

# Load reference data
data_input = torch.as_tensor(
    np.loadtxt(f"data/inputs_Rx={Rx}.csv", delimiter=",")
)
data_output = torch.as_tensor(
    np.loadtxt(f"data/outputs_Rx={Rx}.csv", delimiter=","))

# Create collocation points
points = L * qmc.LatinHypercube(d=2).random(N**2)
Ry = R**2 / Rx
mask = (((points[:, 0] ** 2) / (Rx**2)) + ((points[:, 1] ** 2) / (Ry**2))) > 1
collocation = torch.tensor(points[mask], requires_grad=True)

#additional collocation points
border = L * qmc.LatinHypercube(d=2).random(8*N**2)
upperRy = Ry + 0.3 
upperRx = Rx + 0.3
bordermask = (((border[:, 0] ** 2) / (upperRx**2)) + ((border[:, 1] ** 2) / (upperRy**2))) < 1
bordermask2 = ((((border[bordermask][:, 0] ** 2) / (Rx**2)) + ((border[bordermask][:, 1] ** 2) / (Ry**2))) > 1)
bordermask3 = (border[bordermask][bordermask2][:, 0] <  0.20)
#bordermask4 = (border[bordermask][bordermask2][bordermask3][:, 1] <  0.95)
addi_collo = torch.tensor(border[bordermask][bordermask2][bordermask3], requires_grad=True).double()
complete_collo = torch.cat((collocation, addi_collo), 0)

# Boundary points
x_top = torch.linspace(0, L, N, requires_grad=True).double()
y_top = L * torch.ones((N, 1), requires_grad=True).double()
top = torch.column_stack([x_top, y_top]).double()

NN = int(N * (L - Rx) / L)
x_bottom = torch.linspace(Rx, L, NN).double()
y_bottom = torch.zeros((NN, 1)).double()
bottom = torch.column_stack([x_bottom, y_bottom]).double()

NN = int(N * (L - Ry) / L)
x_left = torch.zeros((NN, 1)).double()
y_left = torch.linspace(Ry, L, NN).double()
left = torch.column_stack([x_left, y_left]).double()

x_right = L * torch.ones((N, 1)).double()
y_right = torch.linspace(0, L, N).double()
right = torch.column_stack([x_right, y_right]).double()

phi = np.linspace(0, 0.5 * np.pi, int(N * np.pi * Rx / L))
x_hole = torch.tensor(Rx * np.cos(phi), requires_grad=True)
y_hole = torch.tensor(Ry * np.sin(phi), requires_grad=True)
n_hole = torch.tensor(np.stack([-Ry * np.cos(phi), -Rx * np.sin(phi)]).T)
n_hole = n_hole / torch.linalg.norm(n_hole, axis=1)[:, None]
hole = torch.column_stack([x_hole, y_hole]).double()

# Visualize geometry
with torch.no_grad():
    mode = "markers"
    gray = dict(color="#C9C5BC")
    green = dict(color="#006561")
    black = dict(color="black")
    fig = ff.create_quiver(
        hole[:, 0], hole[:, 1], n_hole[:, 0], n_hole[:, 1], marker=black
    )
    fig.add_trace(
        go.Scatter(x=collocation[:, 0], y=collocation[:, 1], mode=mode, marker=gray)
    )
    fig.add_trace(go.Scatter(x=top[:, 0], y=top[:, 1], mode=mode, marker=green))
    fig.add_trace(go.Scatter(x=bottom[:, 0], y=bottom[:, 1], mode=mode, marker=green))
    fig.add_trace(go.Scatter(x=left[:, 0], y=left[:, 1], mode=mode, marker=green))
    fig.add_trace(go.Scatter(x=right[:, 0], y=right[:, 1], mode=mode, marker=green))
    fig.add_trace(go.Scatter(x=hole[:, 0], y=hole[:, 1], mode=mode, marker=black))
    fig.layout.yaxis.scaleanchor = "x"
    fig.update_layout(
        template="none",
        width=400,
        height=400,
        margin=dict(l=0, r=0, b=0, t=0),
        showlegend=False,
    )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    wandb.log({"problem_visualization": fig})
    fig.show()

## The ANN model that approximates the displacement field

An ANN might be considered as a generic function approximator. In this case, it should approximated the function $u: \mathcal{R}^2 \rightarrow \mathcal{R}^2$ with five hidden layers having 20 neurons each.

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden_layer1 = torch.nn.Linear(2, hn)
        self.hidden_layer2 = torch.nn.Linear(hn, hn)
        self.hidden_layer3 = torch.nn.Linear(hn, hn)
        self.hidden_layer4 = torch.nn.Linear(hn, hn)
        self.hidden_layer5 = torch.nn.Linear(hn, hn)
        # self.hidden_layer6 = torch.nn.Linear(hn, hn)
        # self.hidden_layer7 = torch.nn.Linear(hn, hn)
        # self.hidden_layer8 = torch.nn.Linear(hn, hn)
        self.output_layer = torch.nn.Linear(hn, 2)

    def forward(self, inputs):
        layer1_out = torch.tanh(self.hidden_layer1(inputs))
        layer2_out = torch.tanh(self.hidden_layer2(layer1_out))
        layer3_out = torch.tanh(self.hidden_layer3(layer2_out))
        layer4_out = torch.tanh(self.hidden_layer4(layer3_out))
        layer5_out = torch.tanh(self.hidden_layer5(layer4_out))
        # layer6_out = torch.tanh(self.hidden_layer5(layer5_out))
        # layer7_out = torch.tanh(self.hidden_layer5(layer6_out))
        # layer8_out = torch.tanh(self.hidden_layer5(layer7_out))
        output = self.output_layer(layer5_out)
        return output


net = Net()

## The physics

We want to solve linear elasticity on the domain, which means ultimately that we want to minimize the residual of the following PDE 
$$\frac{\partial \sigma_{11}}{\partial x_1} + \frac{\partial \sigma_{12}}{\partial x_2} - b_1 = 0$$
$$\frac{\partial \sigma_{21}}{\partial x_1} + \frac{\partial \sigma_{22}}{\partial x_2} - b_2 = 0$$
with stress 
$$ \sigma_{ij} = 2\mu \varepsilon_{ij} + \lambda \varepsilon_{kk} \delta_{ij} $$
and strain 
$$ \varepsilon_{ij} = \frac{1}{2} \left( \frac{\partial u_i}{\partial x_j} +  \frac{\partial u_j}{\partial x_i}\right).$$

In [ ]:
def epsilon(x):
    # Compute deformation gradient
    dudx = torch.func.jacrev(net)(x)
    return 0.5 * (dudx + dudx.T)


def sigma(x):
    # Compute (small deformation) strain
    eps = epsilon(x)
    # Compute linear elastic strain (assuming plane strain)
    return 2.0 * MU * eps + LBD * torch.trace(eps) * torch.eye(2)


def pde_residual(x):
    # Compute stress gradient
    dsdx = torch.func.jacrev(sigma)(x)
    # Momentum balance in x direction
    residual_x = dsdx[0, 0, 0] + dsdx[0, 1, 1] - B0
    # Momentum balance in y direction
    residual_y = dsdx[1, 0, 0] + dsdx[1, 1, 1]
    return residual_x, residual_y

In [ ]:
mse = torch.nn.MSELoss()


def compute_physics_losses(complete_collo):
    # pde
    res_x, res_y = torch.vmap(pde_residual)(complete_collo)
    zeros = torch.zeros_like(res_x)
    pde_error = mse((res_x), zeros) + mse((res_y), zeros)

    # left boundary
    pred_left = net(left)
    bc_left = torch.zeros_like(pred_left[:, 0])
    left_error = mse(pred_left[:, 0], bc_left)

    # right boundary
    pred_right = net(right)
    bc_right = EPS0 * L * torch.ones_like(pred_right[:, 0])
    right_error = mse(pred_right[:, 0], bc_right)

    # bottom boundary
    pred_bottom = net(bottom)
    bc_bottom = torch.zeros_like(pred_bottom[:, 1])
    bottom_error = mse(pred_bottom[:, 1], bc_bottom)

    # top boundary
    pred_top = net(top)
    bc_top = RATIO * EPS0 * L * torch.ones_like(pred_top[:, 1])
    top_error = mse(pred_top[:, 1], bc_top)

    # hole boundary
    stress_hole = torch.vmap(sigma)(hole)
    traction = torch.einsum("...ij,...j->...i", stress_hole, n_hole)
    zeros = torch.zeros_like(traction[:, 0])
    hole_error = mse(traction[:, 0], zeros) + mse(traction[:, 1], zeros)

    return (left_error, right_error, bottom_error, top_error, hole_error, pde_error)

## Training 

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)
history = []

print("Starting with Adam optimizer...")
for epoch in tqdm(range(epochs)):
    # Permutation to shuffle collocation points randomly in each epoch
    permutation = torch.randperm(complete_collo.size()[0])

    for i in range(0, complete_collo.size()[0], batch_size):
        indices = permutation[i : i + batch_size]
        collo = complete_collo[indices]
        optimizer.zero_grad()

        # Compute physics losses
        left_l, right_l, bottom_l, top_l, hole_l, pde_l = compute_physics_losses(collo)

        # Get samples from reference solution
        samples = torch.randperm(data_output.size()[0])[::100]
        # Reference solutions
        s_data = data_output[samples, 0:3]
        e_data = data_output[samples, 4:7]
        u_data = data_output[samples, 7:10]
        # Predictions
        s_pred = torch.vmap(sigma)(data_input[samples, 0:2])
        e_pred = torch.vmap(epsilon)(data_input[samples, 0:2])
        u_pred = net(data_input[samples, 0:2])
        # Compute data losses
        ds_xx = mse(s_data[:, 0], s_pred[:, 0, 0])
        ds_yy = mse(s_data[:, 1], s_pred[:, 1, 1])
        ds_xy = mse(s_data[:, 2], s_pred[:, 0, 1])
        stress_l = ds_xx + ds_yy + ds_xy
        de_xx = mse(e_data[:, 0], e_pred[:, 0, 0])
        de_yy = mse(e_data[:, 1], e_pred[:, 1, 1])
        de_xy = mse(e_data[:, 2], e_pred[:, 0, 1])
        strain_l = de_xx + de_yy + de_xy
        du_x = mse(u_data[:, 0], u_pred[:, 0])
        du_y = mse(u_data[:, 1], u_pred[:, 1])
        disp_l = du_x + du_y

        # Aggregate losses
        dirichlet_losses = left_l + right_l + bottom_l + top_l
        loss = (
            dirichlet_losses
            + W_NEU * hole_l
            + W_PDE * pde_l
            + W_STRESS * stress_l
            + W_DISP * disp_l
        )

        # Make optimization step after batch
        loss.backward(retain_graph=True)
        optimizer.step()

    # Make scheduler step after full epoch
    scheduler.step()
    
    # append loss to history (=for plotting)
    with torch.autograd.no_grad():
        history.append(float(loss.data))


print("Switching to LBFGS optimizer for fine tuning...")
optimizer = torch.optim.LBFGS(net.parameters())

for epoch in tqdm(range(epochs2)):
    def closure():
        optimizer.zero_grad()

        # Compute physics losses
        left_l, right_l, bottom_l, top_l, hole_l, pde_l = compute_physics_losses(collo)

        # Get samples from reference solution
        samples = torch.randperm(data_output.size()[0])[::100]
        # Reference solutions
        s_data = data_output[samples, 0:3]
        e_data = data_output[samples, 4:7]
        u_data = data_output[samples, 7:10]
        # Predictions
        s_pred = torch.vmap(sigma)(data_input[samples, 0:2])
        e_pred = torch.vmap(epsilon)(data_input[samples, 0:2])
        u_pred = net(data_input[samples, 0:2])
        # Compute data losses
        ds_xx = mse(s_data[:, 0], s_pred[:, 0, 0])
        ds_yy = mse(s_data[:, 1], s_pred[:, 1, 1])
        ds_xy = mse(s_data[:, 2], s_pred[:, 0, 1])
        stress_l = ds_xx + ds_yy + ds_xy
        de_xx = mse(e_data[:, 0], e_pred[:, 0, 0])
        de_yy = mse(e_data[:, 1], e_pred[:, 1, 1])
        de_xy = mse(e_data[:, 2], e_pred[:, 0, 1])
        strain_l = de_xx + de_yy + de_xy
        du_x = mse(u_data[:, 0], u_pred[:, 0])
        du_y = mse(u_data[:, 1], u_pred[:, 1])
        disp_l = du_x + du_y

        # Aggregate losses
        dirichlet_losses = left_l + right_l + bottom_l + top_l
        loss = (
            dirichlet_losses
            + W_NEU * hole_l
            + W_PDE * pde_l
            + W_STRESS * stress_l
            + W_DISP * disp_l
        )

        # Make optimization step after batch
        loss.backward(retain_graph=True)
        # append loss to history (=for plotting)
        with torch.autograd.no_grad():
            history.append(float(loss.data))
        return loss
    optimizer.step(closure)


plt.plot(history, c='g', label='train', linewidth=2.0)
plt.yscale("log")
plt.title("Training")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend()
plt.show()

    # with torch.autograd.no_grad():
    #     wandb.log(
    #         {
    #             "loss": loss,
    #             "left_loss": left_l,
    #             "right_loss": right_l,
    #             "bottom_loss": bottom_l,
    #             "top_loss": top_l,
    #             "hole_loss": hole_l,
    #             "pde_loss": pde_l,
    #             "du_x": du_x,
    #             "du_y": du_y,
    #             "ds_xx": ds_xx,
    #             "ds_xy": ds_xy,
    #             "ds_yy": ds_yy,
    #             "de_xx": de_xx,
    #             "de_xy": de_xy,
    #             "de_yy": de_yy,
    #         }
    #     )

## Visualization of results

In [ ]:
# Create a validation domain different from the training domain
val_x, val_y = np.meshgrid(np.linspace(0, L, 50), np.linspace(0, L, 50))
val_domain = np.vstack([val_x.ravel(), val_y.ravel()]).T
mask = (
    ((val_domain[:, 0] ** 2) / (Rx**2)) + ((val_domain[:, 1] ** 2) / (Ry**2))
) > 1
val = torch.tensor(val_domain[mask], requires_grad=True)

# Compute model predictions on the validation domain
disp = net(val)
def_val = val + disp
stress = torch.vmap(sigma)(val)
mises = torch.sqrt(
    stress[:, 0, 0] ** 2
    + stress[:, 1, 1] ** 2
    - stress[:, 0, 0] * stress[:, 1, 1]
    + 3 * stress[:, 0, 1] ** 2
)
# print([loss.item() for loss in compute_les(val)])


@torch.no_grad()
def make_plot(x, y, variable, title, cmap=sequential.Viridis, size=8.0):
    fig = go.Figure()

    # Plot boundaries
    m = dict(color="black")
    fig.add_trace(go.Scatter(x=top[:, 0], y=top[:, 1], mode="lines", marker=m))
    fig.add_trace(go.Scatter(x=bottom[:, 0], y=bottom[:, 1], mode="lines", marker=m))
    fig.add_trace(go.Scatter(x=left[:, 0], y=left[:, 1], mode="lines", marker=m))
    fig.add_trace(go.Scatter(x=right[:, 0], y=right[:, 1], mode="lines", marker=m))
    fig.add_trace(go.Scatter(x=hole[:, 0], y=hole[:, 1], mode="lines", marker=m))

    # Plot variable values
    m = dict(color=variable, colorscale=cmap, size=size, colorbar=dict(thickness=10))
    fig.add_trace(go.Scatter(x=x, y=y, marker=m, mode="markers"))

    # plot settings
    fig.layout.yaxis.scaleanchor = "x"
    fig.update_layout(
        template="none", width=400, height=400, title=title, showlegend=False
    )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    wandb.log({f"chart_{title}": fig})
    fig.show()


# Compute stress data error
s_data = data_output[:, 0:3]
s_pred = torch.vmap(sigma)(data_input[:, 0:2])
ds_xx = s_data[:, 0] - s_pred[:, 0, 0]
ds_yy = s_data[:, 1] - s_pred[:, 1, 1]
ds_xy = s_data[:, 2] - s_pred[:, 0, 1]

# Compute strain data error
e_data = data_output[:, 4:7]
e_pred = torch.vmap(epsilon)(data_input[:, 0:2])
de_xx = e_data[:, 0] - e_pred[:, 0, 0]
de_yy = e_data[:, 1] - e_pred[:, 1, 1]
de_xy = e_data[:, 2] - e_pred[:, 0, 1]

res_x, res_y = torch.vmap(pde_residual)(collocation)

# Plot PDE residual
make_plot(*collocation.T, res_x, "Residual x")
make_plot(*collocation.T, res_y, "Residual y")

# Hole residual
resh_x, resh_y = torch.vmap(pde_residual)(hole)
make_plot(*hole.T, resh_x, "Residual x")
make_plot(*hole.T, resh_y, "Residual y")

# # Hole error 
# strain_hole = torch.vmap(epsilon)(hole)
# stress_hole = torch.vmap(sigma)(hole)
# normal_strain = torch.einsum("...ij,...j->...i", strain_hole, n_hole)
# normal_stress = torch.einsum("...ij,...j->...i", stress_hole, n_hole)

# Plot stress errors
cmap = sequential.RdBu_r
make_plot(*data_input[:, 0:2].T, ds_xx, "Stress error xx", size=2.0, cmap=cmap)
make_plot(*data_input[:, 0:2].T, ds_yy, "Stress error yy", size=2.0, cmap=cmap)
make_plot(*data_input[:, 0:2].T, ds_xy, "Stress error xy", size=2.0, cmap=cmap)

# Plot strain errors
cmap = sequential.RdBu_r
make_plot(*data_input[:, 0:2].T, de_xx, "Strain error xx", size=2.0, cmap=cmap)
make_plot(*data_input[:, 0:2].T, de_yy, "Strain error yy", size=2.0, cmap=cmap)
make_plot(*data_input[:, 0:2].T, de_xy, "Strain error xy", size=2.0, cmap=cmap)

# Plot stresses
make_plot(*def_val.T, stress[:, 0, 0], "Stress xx")
make_plot(*def_val.T, stress[:, 0, 1], "Stress xy")
make_plot(*def_val.T, stress[:, 1, 1], "Stress yy")
make_plot(*def_val.T, mises, "Mises stress")

# Plot displacements
make_plot(*def_val.T, disp[:, 0], "Displacement in x", cmap=sequential.Inferno)
make_plot(*def_val.T, disp[:, 1], "Displacement in y", cmap=sequential.Inferno)

# Finish tracking
wandb.finish()